In [56]:
# Auxiliar imports
import sys, os, time
import matplotlib.pyplot as plt
import numpy as np

# Gym imports
import gym
from gym.vector import SyncVectorEnv

# PyTorch imports
import torch
from torch import nn, optim

# Custom imports
sys.path.append(os.path.abspath('..')) # Add parent directory to path
import ppo_network, ppo_wrapper, importlib
importlib.reload(ppo_network) # Prevents caching issues with notebooks
from ppo_network import PPONetwork
importlib.reload(ppo_wrapper) # Prevents caching issues with notebooks
from ppo_wrapper import PPOWrapper

import hp_tuner
importlib.reload(hp_tuner) # Prevents caching issues with notebooks
from hp_tuner import HPTuner

In [57]:
# CartPole environment
env_id = 'CartPole-v1'
max_episode_steps = 1024
num_envs = 16

env_kwargs = {
    'id': env_id,
    'max_episode_steps': max_episode_steps,
}

# Create vectorized environment
envs_vector = SyncVectorEnv([lambda: gym.make(**env_kwargs)] * num_envs)
states, infos = envs_vector.reset()

In [58]:
# Network
input_dims = 4
output_dims = 2
shared_hidden_dims = [512, 256, 128]
policy_hidden_dims = [64, 64]
value_hidden_dims = [64, 64]
activation = nn.ReLU

network_kwargs = {
    'input_dims': input_dims,
    'output_dims': output_dims,
    'shared_hidden_dims': shared_hidden_dims,
    'policy_hidden_dims': policy_hidden_dims,
    'value_hidden_dims': value_hidden_dims,
    'activation': activation,
}

network = PPONetwork(**network_kwargs)

# Test forward pass
now = time.time()
for _ in range(100):
    states_tensor = torch.tensor(states, dtype=torch.float32)
    policy, value = network(states_tensor)
    
    actions_dist = torch.distributions.Categorical(logits=policy)
    actions = actions_dist.sample().numpy()
    
    states, rewards, dones, truncateds, infos = envs_vector.step(actions)
    #print(dones)

print(
    f'Elapsed time: per vectorized env: {(time.time() - now)/num_envs:.2f} s'
    )

Elapsed time: per vectorized env: 0.00 s


/Users/axelsolhall/miniconda3/envs/PPOgym/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [59]:
# PPO hyperparameters
lr = 3e-4
final_lr = 1e-6

gamma = 0.99
lam = 0.90

clip_eps = 0.25
final_clip_eps = 0.01

value_coef = 0.7

entropy_coef = 0.1
final_entropy_coef = 0.01

batch_size = 256
batch_epochs = 10
batch_shuffle = True

iterations = 1024

truncated_reward = 5

debug_prints = False

ppo_kwargs = {
    'num_envs': num_envs,
    'lr': lr,
    'final_lr': final_lr,
    'gamma': gamma,
    'lam': lam,
    'clip_eps': clip_eps,
    'final_clip_eps': final_clip_eps,
    'value_coef': value_coef,
    'entropy_coef': entropy_coef,
    'final_entropy_coef': final_entropy_coef,
    'batch_size': batch_size,
    'batch_epochs': batch_epochs,
    'batch_shuffle': batch_shuffle,
    'iterations': iterations,
    'truncated_reward': truncated_reward,
    'debug_prints': debug_prints,   
}

#ppo_wrapper = PPOWrapper(envs_vector, network, **ppo_kwargs)

#ppo_wrapper.train(generations=1)


In [60]:
hp_optimizer = HPOptimizer(
    env_kwargs=env_kwargs,
    num_envs=num_envs,
    network_class = PPONetwork,
    network_kwargs=network_kwargs,
    ppo_class=PPOWrapper,
    ppo_kwargs=ppo_kwargs,
)

parameters = [
    'clip_eps', 
    'value_coef', 
    'entropy_coef', 
    'batch_size', 
    'batch_epochs',
    ]

# evolutions = hp_optimizer.optimize_hyperparameters(
#     parameters, generations=50, num_trials = 8,
#     )

hp_optimizer.evolution_video(
    generations=250, video_folder = 'videos', increments=25, max_frames=120
    )

Running evolution with generations: [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250]
Video saved to videos/evolution_video_250_gens_9ecf97d0.mp4
